<a href="https://colab.research.google.com/github/GerryMasaka/NextWordPrediction/blob/main/Next_Word_Prediction_ICS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!python -m spacy download en_core_web_md

In [ ]:
#import spacy

In [ ]:
#embedding_file=spacy.load('en_core_web_md')

In [ ]:
#embedding_file('want').vector.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
#!pip install nlp

In [ ]:
#from nlp import load_dataset
from nltk.tokenize import word_tokenize,sent_tokenize

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
import nltk
import math
import json
import tensorflow_hub as hub
import spacy

In [ ]:
nltk.download('punkt')
nltk.download('gutenberg')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [ ]:
from tensorflow.keras.layers import SimpleRNN,GRU,LSTM,Dense,Embedding,InputLayer,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import CategoricalCrossentropy
from tensorflow.keras.losses import CategoricalCrossentropy,categorical_crossentropy
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from nltk.corpus import gutenberg

In [ ]:
DIM=10

In [ ]:
data=open("0001a.txt")
data=data.read()

FileNotFoundError: ignored

In [ ]:
data

In [ ]:
corpus_list=[]
corpus=sent_tokenize(data)
for sent in corpus:
  corpus_list.append([sent])

In [ ]:
corpus_list

In [ ]:
len(corpus_list)

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X = vectorizer.fit_transform(corpus)

In [ ]:
X.shape

In [ ]:
vocab=vectorizer.get_feature_names_out()

In [ ]:
#vocab

In [ ]:
len(vocab)

In [ ]:
embeddings_array=X.toarray().T

In [ ]:
vocab[458]

In [ ]:
embeddings_array[458]

In [ ]:
pca=PCA(n_components=DIM)

In [ ]:
dense_embeddings=pca.fit_transform(embeddings_array)

In [ ]:
dense_embeddings[458]

In [ ]:
embeddings_dict={}

In [ ]:
for i in range(len(vocab)):
  embeddings_dict[vocab[i]]=dense_embeddings[i].tolist()

In [ ]:
#embeddings_dict

In [ ]:
seq_len=10
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])
x=tokenizer.texts_to_sequences([data])
#x

In [ ]:
#x[0]

In [ ]:
len(x[0])

In [ ]:
#tokenizer.get_config()

In [ ]:
vocab_dict=json.loads(tokenizer.get_config().get('word_index'))

In [ ]:
vocab_dict

In [ ]:
vals=list(vocab_dict.values())
keys=list(vocab_dict.keys())
#keys[vals.index(1)]

In [ ]:
vocab_size=len(keys)

In [ ]:
vocab_size

In [ ]:
embedding_array=np.zeros((vocab_size+1,DIM))
embedding_array.shape

In [ ]:
for word,i in tokenizer.word_index.items():
  if word in embeddings_dict:
    embedding_vector=embeddings_dict[word]
  if embedding_vector is not None:
    embedding_array[i]=embedding_vector

In [ ]:
"""for word,i in tokenizer.word_index.items():
  embedding_vector=embedding_file(word).vector
  if embedding_vector is not None:
    embedding_array[i]=embedding_vector"""

In [ ]:
#embedding_array[7306]

In [ ]:
len(x[0])

In [ ]:
x[0][0:15]

In [ ]:
inner_list=[]
X={}
corpus_length=len(x[0])-seq_len
k=0

for i in range(0,corpus_length,seq_len):
  inner_list.clear()
  for j in range(i,i+seq_len):
    inner_list.append(x[0][j])
  X[k]=inner_list[:]
  k=k+1

In [ ]:
#list(X.values())
x_train=[]
y_train=[]
for it in list(X.values()):
  x_train.append(it[0:-1])
  y_train.append(it[-1])

In [ ]:
x_train[3] 

In [ ]:
len(x_train)

In [ ]:
X=np.array(x_train)
#input_seq.shape
y=np.array(y_train).reshape(len(y_train),1)
#target=tf.reshape(tf.constant(y_train),[len(y_train),1])

In [ ]:
y.shape

NameError: ignored

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [ ]:
X_train.shape

In [ ]:
y_train

In [ ]:
input_seq,test_seq,y_true,y_hat=tf.constant(X_train),tf.constant(X_test),tf.constant(y_train),tf.constant(y_test)

In [ ]:
input_seq

In [ ]:
y_true

In [ ]:
tf.reduce_max(input_seq)+1

In [ ]:
input=Input(shape=(seq_len-1,))
x=Embedding(input_dim=vocab_size+1,input_length=seq_len-1,output_dim=20,trainable=True)(input)
x=LSTM(256,return_sequences=True)(x)
x=LSTM(128,return_sequences=True)(x)
x=LSTM(64)(x)
x=Dense(32)(x)
output=Dense(vocab_size,activation="softmax")(x)
model=Model(input,output)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(0.01),loss=tf.keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])

In [ ]:
model.fit(input_seq,y_true,epochs=10)

In [ ]:
model.evaluate(test_seq,y_hat)

In [ ]:
input_text=["she was such a good person"]
#input_text=["he was such a good person and honestly speaking he did make good use of his"]
#input_text=["can you please"]
#input_text=["marriage is such a beautiful"]
toks=tokenizer.texts_to_sequences(input_text)
ins=pad_sequences(toks,maxlen=seq_len-1)
input_tensor=tf.constant(ins)
input_tensor

In [ ]:
pred=model.predict(input_tensor)
keys[vals.index(np.argmax(pred))]

In [ ]:
input_text[0]=input_text[0]+' '+keys[vals.index(np.argmax(pred))]
input_text[0]

In [ ]:
for i in range(10):
  toks=tokenizer.texts_to_sequences(input_text)
  ins=pad_sequences(toks,maxlen=seq_len-1)
  input_tensor=tf.constant(ins)
  pred=model.predict(input_tensor)
  input_text[0]=input_text[0]+' '+keys[vals.index(np.argmax(pred))]

print(input_text)

In [ ]:
sort_pred=np.argsort(pred[0])
pred_words=np.flip(sort_pred[-20:])
for index in pred_words:
  print(keys[vals.index(index)])